Setpinski [19], wavenumber algorithm [16], [17]

1. Split the data pre focal region and post focal region

2. Prefocal consists of electro-mechanical impulse response and excitation pulse ($s_\text{pre}(x_n,t)$ and $h_\text{pre}(t)$) is reversed in axial direction

3. 2D FFT on all data to get $S(k_x, \omega), H(\omega), P(\omega)$. Far field directivity pattern of VS is assumed to be $A=jinc^2(k_xa)$ , where $a$ is radius of the flat circular virtual source that can be determined from the simulated beam profile.

(see: 

https://en.wikipedia.org/wiki/Sombrero_function, 

https://en.wikipedia.org/wiki/Bessel_function#Bessel_functions_of_the_first_kind:_J%CE%B1, 

https://www.johndcook.com/blog/2012/02/02/how-to-compute-jincx/)

4. Image reconstruction
$$F(k_x, k_z)=\mathcal{S}\left\{\exp\left[j(\sqrt{4k^2-k_x^2}-2k)z_c\right]A^*\omega^2H^*(\omega)P^*(\omega)S(k_x,\omega)\right\}$$

where $\mathcal{S}\{\cdot\}$ is the Stolt transformation. $z_c$ is the perpendicular distance from the transducer to the midpoint of the ROI. $k=\omega/c$. 

Stolt migration:

http://sepwww.stanford.edu/sep/prof/iei/omk/paper_html/node11.html

https://www.math.ucdavis.edu/~saito/data/sonar/stolt.pdf

5. 2D IFFT on $F_\text{pre}$ and $F_\text{post}$ to spatial domain, $F_\text{pre}$ flipped axially, and then stitched together

# TD-SAFT

Image reconstruction is done by delaying the recorded echo signals according to the relative positions of the image pixel and the transducer, which is then followed by a coherent summation. Each point $(x,z)$ in the image is brought into focus by applying appropriate time-delays to the recorded echo data $s(x_n,t)$ for all the transducer positions $x_n(n\in\{0\ldots L-1\})$ in the synthetic aperture and then performing summation as follows,
$$f_\text{TD}(x,z)=\sum_{n=0}^{L-1}s\left(x_n, \dfrac{2}{c_0}\sqrt{(x-x_n)^2+z^2}\right)$$

where $c_0$ is the speed of sound, and $f_\text{TD}(x,z)$ is the reconstructed TD-SAFT image.

The time-domain virtual point source SAFT (TD-SAFT-VPS), treats the focus of the transducer as a point source where the above equation is applied in the pre-focal and post-focal regions separately. In the pre-focal region, the recorded echo data is flipped in the axial direction and then DAS is applied. In the post-focal region, DAS is applied to the recorded echo data without changing the orientation of the data. After applying DAS, the pre-focal region is flipped back to its original orientation before joining it with the post-focal region. **In addition, the image intensity of each point can be normalized by dividing the square root of the number of scanlines used at each point.**

In [68]:
import numpy as np
from os import getcwd
from os.path import join, dirname
import pickle
import multiprocessing as mp
global min_step, c_0
FOLDER_NAME = "1D-15FOC3in"
FOCAL_DEPTH = 0.0381  # 1.5 inch in metres
min_step = 4e-4
c_0 = 1498  # water
c_1 = 6420  # aluminium


# Load voltage data
def load_arr(output_folder= join(dirname(getcwd()), "data", FOLDER_NAME)):
    ftarr = join(output_folder, "tarr.pkl")
    fvarr = join(output_folder, "varr.pkl")
    with open(ftarr, 'rb') as rd:
        tarr = pickle.load(rd)
    with open(fvarr, 'rb') as rd:
        varr = pickle.load(rd)
    return tarr, varr


tarr, varr = load_arr()
tarr = tarr[:, 0, :]
tstep = np.mean(tarr[1:, :] - tarr[:-1, :])
varr = varr[:, 0, :]


# express z in terms of indices
def t(z):
    return (2*z/c_0)//tstep


# check if elements of tarr differ along axis=1
eq = tarr[:, 1:] == tarr[:, :-1]
con = False in eq


def delay_t(x, z):
    return lambda xn : (2/c_0)*np.sqrt((x-xn)**2+z)


def smm(xi, T, V, xarr, tstep):
    x = xarr[xi]  # metres
    for ti in range(len(T)):
        z = T[ti]*c_0/2  # metres
        dt = delay_t(x, z)  # anon function dependent on xn
        s = 0  # summation
        for xn in xarr:
            s += V(int(dt(xn)//tstep, xi))

    return s


def SAFT(T, V):
    T = T[:, 0]  # time values, 1D
    tstep = np.mean(T[1:, :] - T[:-1, :])
    xarr = np.arange(0, np.shape(V)[1], 1)*min_step  # lateral distance, 1D
    pool = mp.Pool(mp.cpu_count())
    results = pool.starmap(smm, [(xi, T=T, V=V, xarr=xarr, tstep=tstep) for xi in range(len(xarr))])
    pool.close()
    return results
                   
# Split data into pre and post focal depth
FD = int((2*FOCAL_DEPTH/c_0)//tstep) # FOCAL_DEPTH_IND
PRE_V = np.flip(varr[:FD+1, :], axis=0)
PRE_T = np.flip(tarr[:FD+1])
POST_V = varr[FD+1:, :]
POST_T = tarr[FD+1:]

arr = SAFT(PRE_T, PRE_V)

SyntaxError: invalid syntax (<ipython-input-68-07766c906906>, line 24)

In [45]:
out_arr = np.empty(np.shape(varr)[1], dtype=float)
for x in range(len(xarr)):
    for z in range(len(varr)):
        s = 0
        for n in range(len(xarr)):
            xn = xarr[n]
            t = int((2/c_0)*np.sqrt((x*min_step - xn)**2 + z**2)//tstep)
            s += varr[t, xn]
        out_arr[z, x] = s

TypeError: only integer scalar arrays can be converted to a scalar index

5086.0